<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/343_WDO_Risk_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Workforce Development Orchestrator — Automation Risk Analysis Utilities

This module is responsible for **quantifying how exposed work is to automation** and translating that exposure into **actionable, explainable insights**. It is one of the most critical components of the orchestrator because it directly informs reskilling priorities, role evolution decisions, and executive risk conversations.

Importantly, this entire layer is **rules-based and transparent**. No LLMs are involved. Every decision can be inspected, adjusted, and justified.

---

## 1. Risk Categorization Is Policy-Driven, Not Hard-Coded

The `categorize_task_by_risk` function classifies tasks as **low, medium, or high automation risk** using thresholds defined in configuration.

This means:

* Leadership can change what “high risk” means
* Sensitivity can be tuned without rewriting code
* Risk policy is explicit and auditable

This small function quietly enforces a major design principle:
**business policy controls system behavior.**

---

## 2. Role-Level Risk Analysis Reflects How Work Is Actually Organized

The `analyze_role_automation_risk` function evaluates automation exposure at the **role level**, not just the task level.

It does four important things:

### 📊 Aggregates Task Risk

Tasks are grouped into:

* High-risk
* Medium-risk
* Low-risk

This makes it easy to see *where* automation pressure is concentrated.

---

### ⚖️ Computes an Overall Risk Score

Rather than cherry-picking worst cases, the function calculates a **weighted average risk score** across all tasks.

This avoids alarmism and supports balanced decision-making:

* A role with one risky task is not treated the same as a role dominated by automation-prone work.

---

### 👥 Identifies Affected Employees

Employees are explicitly attached to role risk outcomes.

This allows the agent to move from:

> “This role is risky”
> to
> “These people are impacted”

That shift is essential for ethical workforce planning.

---

### 💡 Generates Human-Readable Recommendations

Recommendations are not generic. They are conditional:

* High-risk task concentration triggers upskilling guidance
* Medium or high overall risk triggers role evolution review

These recommendations are **derived from facts**, not intuition.

---

## 3. Employee-Level Risk Is Derived, Not Recomputed

The `analyze_employee_automation_risk` function does something subtle and very smart:

It **reuses role-level analysis** rather than inventing a new employee-specific model.

This ensures:

* Consistency between role and employee narratives
* No conflicting conclusions
* Simpler reasoning paths

Employee analysis adds context (name, ID) — not new logic.

---

## 4. Organization-Wide Analysis Is Composable

The final two functions scale the same logic cleanly:

* `analyze_all_roles_automation_risk`
* `analyze_all_employees_automation_risk`

Both:

* Iterate deterministically
* Apply the same core logic everywhere
* Produce uniform analysis artifacts

This makes results:

* Comparable across roles
* Aggregatable for dashboards
* Suitable for trend tracking over time

---

## 5. Explainability Is Built In, Not Added Later

Every analysis output includes:

* Risk scores
* Risk levels
* Task IDs driving those scores
* Employee impact lists
* Plain-language recommendations

This allows the orchestrator to confidently answer:

> **“Why is this role considered at risk?”**

Without referencing a model, a prompt, or probabilistic reasoning.

---

## Why This Design Works for Executives

From a leadership perspective, this module ensures:

* Automation risk is **measured**, not guessed
* Thresholds are **visible and adjustable**
* Recommendations are **defensible**
* Workforce impact is **human-centered**, not abstract

From an engineering perspective, it ensures:

* Deterministic behavior
* Easy testing
* Clear failure modes
* Reusability across agents

---

## Architectural Takeaway

This module demonstrates a critical principle of trustworthy AI systems:

> **Risk analysis should be governed by policy, not hidden intelligence.**

By grounding automation risk in explicit scores, configurable thresholds, and explainable aggregation logic, the orchestrator earns the right to recommend reskilling and role evolution — instead of imposing it.




In [ ]:
"""Automation risk analysis utilities for Workforce Development Orchestrator

Following the pattern: Utilities implement, nodes orchestrate.
These utilities analyze automation risk for tasks and roles.
"""

from typing import Dict, List, Any, Optional
from config import WorkforceDevelopmentOrchestratorConfig


def categorize_task_by_risk(
    task: Dict[str, Any],
    config: WorkforceDevelopmentOrchestratorConfig
) -> str:
    """Categorize task by automation risk level"""
    score = task.get("automation_risk_score", 0.0)
    thresholds = config.automation_risk_thresholds

    if score >= thresholds["high"]:
        return "high"
    elif score >= thresholds["medium"]:
        return "medium"
    else:
        return "low"


def analyze_role_automation_risk(
    role_id: str,
    tasks: List[Dict[str, Any]],
    employees: List[Dict[str, Any]],
    config: WorkforceDevelopmentOrchestratorConfig
) -> Dict[str, Any]:
    """Analyze automation risk for a specific role"""
    high_risk_tasks = []
    medium_risk_tasks = []
    low_risk_tasks = []

    for task in tasks:
        risk_level = categorize_task_by_risk(task, config)
        if risk_level == "high":
            high_risk_tasks.append(task["task_id"])
        elif risk_level == "medium":
            medium_risk_tasks.append(task["task_id"])
        else:
            low_risk_tasks.append(task["task_id"])

    # Calculate overall risk score (weighted average)
    if tasks:
        total_score = sum(task.get("automation_risk_score", 0.0) for task in tasks)
        overall_risk_score = total_score / len(tasks)
    else:
        overall_risk_score = 0.0

    # Determine overall risk level
    if overall_risk_score >= config.automation_risk_thresholds["high"]:
        overall_risk_level = "high"
    elif overall_risk_score >= config.automation_risk_thresholds["medium"]:
        overall_risk_level = "medium"
    else:
        overall_risk_level = "low"

    # Get affected employees
    affected_employees = [emp["employee_id"] for emp in employees]

    # Generate recommendations
    recommendations = []
    if high_risk_tasks:
        recommendations.append(
            f"Role has {len(high_risk_tasks)} high-risk tasks that may be automated. "
            "Consider upskilling employees in AI collaboration and automation workflows."
        )
    if overall_risk_level in ["high", "medium"]:
        recommendations.append(
            f"Overall automation risk is {overall_risk_level}. "
            "Review role evolution opportunities to augment rather than replace."
        )

    return {
        "role_id": role_id,
        "high_risk_tasks": high_risk_tasks,
        "medium_risk_tasks": medium_risk_tasks,
        "low_risk_tasks": low_risk_tasks,
        "overall_risk_score": round(overall_risk_score, 2),
        "risk_level": overall_risk_level,
        "affected_employees": affected_employees,
        "recommendations": recommendations
    }


def analyze_employee_automation_risk(
    employee: Dict[str, Any],
    role_tasks: List[Dict[str, Any]],
    config: WorkforceDevelopmentOrchestratorConfig
) -> Dict[str, Any]:
    """Analyze automation risk for a specific employee based on their role's tasks"""
    role_id = employee.get("role_id")

    if not role_tasks:
        return {
            "employee_id": employee["employee_id"],
            "role_id": role_id,
            "high_risk_tasks": [],
            "medium_risk_tasks": [],
            "low_risk_tasks": [],
            "overall_risk_score": 0.0,
            "risk_level": "low",
            "recommendations": []
        }

    # Use role-level analysis
    role_analysis = analyze_role_automation_risk(
        role_id,
        role_tasks,
        [employee],  # Single employee
        config
    )

    # Add employee-specific context
    role_analysis["employee_id"] = employee["employee_id"]
    role_analysis["employee_name"] = employee.get("name", "")

    return role_analysis


def analyze_all_roles_automation_risk(
    roles: List[Dict[str, Any]],
    tasks_by_role: Dict[str, List[Dict[str, Any]]],
    employees_by_role: Dict[str, List[Dict[str, Any]]],
    config: WorkforceDevelopmentOrchestratorConfig
) -> List[Dict[str, Any]]:
    """Analyze automation risk for all roles"""
    analyses = []

    for role in roles:
        role_id = role["role_id"]
        tasks = tasks_by_role.get(role_id, [])
        employees = employees_by_role.get(role_id, [])

        analysis = analyze_role_automation_risk(
            role_id,
            tasks,
            employees,
            config
        )
        analyses.append(analysis)

    return analyses


def analyze_all_employees_automation_risk(
    employees: List[Dict[str, Any]],
    tasks_by_role: Dict[str, List[Dict[str, Any]]],
    config: WorkforceDevelopmentOrchestratorConfig
) -> List[Dict[str, Any]]:
    """Analyze automation risk for all employees"""
    analyses = []

    for employee in employees:
        role_id = employee.get("role_id")
        role_tasks = tasks_by_role.get(role_id, [])

        analysis = analyze_employee_automation_risk(
            employee,
            role_tasks,
            config
        )
        analyses.append(analysis)

    return analyses



# Test automation risk analysis utilities

In [ ]:
"""Test automation risk analysis utilities

Testing Phase 3: Automation Risk Analysis Utilities
Following the pattern: Test utilities before building nodes
"""

from pathlib import Path
from agents.workforce_development_orchestrator.utilities.automation_risk import (
    categorize_task_by_risk,
    analyze_role_automation_risk,
    analyze_employee_automation_risk,
    analyze_all_roles_automation_risk,
    analyze_all_employees_automation_risk
)
from agents.workforce_development_orchestrator.utilities.data_loading import (
    load_tasks,
    load_roles,
    load_employees,
    build_tasks_by_role,
    build_employees_by_role
)
from config import WorkforceDevelopmentOrchestratorConfig


def test_categorize_task_by_risk():
    """Test task risk categorization"""
    config = WorkforceDevelopmentOrchestratorConfig()

    # High risk task
    high_task = {"task_id": "T001", "automation_risk_score": 0.85}
    assert categorize_task_by_risk(high_task, config) == "high"

    # Medium risk task
    medium_task = {"task_id": "T002", "automation_risk_score": 0.55}
    assert categorize_task_by_risk(medium_task, config) == "medium"

    # Low risk task
    low_task = {"task_id": "T003", "automation_risk_score": 0.25}
    assert categorize_task_by_risk(low_task, config) == "low"

    print("✅ test_categorize_task_by_risk: PASSED")


def test_analyze_role_automation_risk():
    """Test role automation risk analysis"""
    config = WorkforceDevelopmentOrchestratorConfig()
    data_dir = Path("agents/data")

    # Load data
    roles = load_roles(data_dir)
    tasks = load_tasks(data_dir)
    employees = load_employees(data_dir)
    tasks_by_role = build_tasks_by_role(tasks)
    employees_by_role = build_employees_by_role(employees)

    # Analyze R001 (HR Coordinator)
    role = roles[0]  # R001
    role_tasks = tasks_by_role.get("R001", [])
    role_employees = employees_by_role.get("R001", [])

    analysis = analyze_role_automation_risk(
        "R001",
        role_tasks,
        role_employees,
        config
    )

    assert "role_id" in analysis
    assert analysis["role_id"] == "R001"
    assert "high_risk_tasks" in analysis
    assert "overall_risk_score" in analysis
    assert "risk_level" in analysis
    assert "affected_employees" in analysis
    assert "recommendations" in analysis

    print("✅ test_analyze_role_automation_risk: PASSED")


def test_analyze_employee_automation_risk():
    """Test employee automation risk analysis"""
    config = WorkforceDevelopmentOrchestratorConfig()
    data_dir = Path("agents/data")

    # Load data
    employees = load_employees(data_dir)
    tasks = load_tasks(data_dir)
    tasks_by_role = build_tasks_by_role(tasks)

    # Analyze E001
    employee = employees[0]  # E001
    role_tasks = tasks_by_role.get(employee["role_id"], [])

    analysis = analyze_employee_automation_risk(
        employee,
        role_tasks,
        config
    )

    assert "employee_id" in analysis
    assert analysis["employee_id"] == "E001"
    assert "role_id" in analysis
    assert "overall_risk_score" in analysis

    print("✅ test_analyze_employee_automation_risk: PASSED")


def test_analyze_all_roles_automation_risk():
    """Test analyzing all roles"""
    config = WorkforceDevelopmentOrchestratorConfig()
    data_dir = Path("agents/data")

    # Load data
    roles = load_roles(data_dir)
    tasks = load_tasks(data_dir)
    employees = load_employees(data_dir)
    tasks_by_role = build_tasks_by_role(tasks)
    employees_by_role = build_employees_by_role(employees)

    analyses = analyze_all_roles_automation_risk(
        roles,
        tasks_by_role,
        employees_by_role,
        config
    )

    assert len(analyses) == len(roles)
    assert all("role_id" in a for a in analyses)
    assert all("overall_risk_score" in a for a in analyses)

    print("✅ test_analyze_all_roles_automation_risk: PASSED")


def test_analyze_all_employees_automation_risk():
    """Test analyzing all employees"""
    config = WorkforceDevelopmentOrchestratorConfig()
    data_dir = Path("agents/data")

    # Load data
    employees = load_employees(data_dir)
    tasks = load_tasks(data_dir)
    tasks_by_role = build_tasks_by_role(tasks)

    analyses = analyze_all_employees_automation_risk(
        employees,
        tasks_by_role,
        config
    )

    assert len(analyses) == len(employees)
    assert all("employee_id" in a for a in analyses)
    assert all("overall_risk_score" in a for a in analyses)

    print("✅ test_analyze_all_employees_automation_risk: PASSED")


if __name__ == "__main__":
    print("=" * 60)
    print("Testing Automation Risk Analysis Utilities (Phase 3)")
    print("=" * 60)
    print()

    test_categorize_task_by_risk()
    test_analyze_role_automation_risk()
    test_analyze_employee_automation_risk()
    test_analyze_all_roles_automation_risk()
    test_analyze_all_employees_automation_risk()

    print()
    print("=" * 60)
    print("✅ All automation risk utility tests passed!")
    print("=" * 60)



# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_008_Workforce_Development_Orchestrator % python3 test_automation_risk_utilities.py
============================================================
Testing Automation Risk Analysis Utilities (Phase 3)
============================================================

✅ test_categorize_task_by_risk: PASSED
✅ test_analyze_role_automation_risk: PASSED
✅ test_analyze_employee_automation_risk: PASSED
✅ test_analyze_all_roles_automation_risk: PASSED
✅ test_analyze_all_employees_automation_risk: PASSED

============================================================
✅ All automation risk utility tests passed!
============================================================
(.venv) micahshull@Micahs-iMac AI_AGENTS_008_Workforce_Development_Orchestrator %